### 环境要求

python 3.12 与最新 langchain 存在兼容问题，所以降级安装3.9版本

```cmd
conda create -n .venv python=3.9
conda activate .venv
```

### 安装依赖
```cmd
conda install ipykernel  // jupyter notebook 需要
conda install python-dotenv -c conda-forge 
conda install langchain -c conda-forge 
conda install langchain-community -c conda-forge
conda install langchain-openai -c conda-forge
conda install openai -c conda-forge
conda install tiktoken -c conda-forge  // 对话字符缓存使用
conda install pandas -c conda-forge   // 读取 csv 数据
```

### 通过 LangChain 使用 OpenAI
三个重要的概念：模型、提示与解释器
#### 模型
从 `langchain.chat_models`导入`OpenAI`的对话模型`ChatOpenAI`，除了`OpenAI` 以外，`chat_models`也集成了其它的对话模型。


In [3]:
from langchain_openai import ChatOpenAI
from langchain.prompts import ChatPromptTemplate

# 配置了环境变量，删除项目内的环境变量配置
# import os
# from dotenv import load_dotenv, find_dotenv
# _ = load_dotenv(find_dotenv())


# # 获取环境变量
# api_key = os.environ["OPENAI_API_KEY_LOCAL"]
# base_url = os.environ["OPENAI_BASE_URL"]

# 这里我们将参数temperature设置为0.0，从而减少生成答案的随机性。
# 如果你想要每次得到不一样的有新意的答案，可以尝试调整该参数。
chat = ChatOpenAI(temperature=0.0)

#### 提示模板
在应用于比较复杂的场景时，提示可能会非常长并且包含涉及许多细节。使用提示模版，可以让我们更为方便地重复使用设计好的提示。
LangChain还提供了提示模版用于一些常用场景。比如自动摘要、问答、连接到SQL数据库、连接到不同的API。通过使用LangChain内置的提示模版，你可以快速建立自己的大模型应用，而不需要花时间去设计和构造提示。

In [2]:
# 编写 prompt
template_string = """把由三个反引号分隔的文本\
翻译成一种{style}风格。\
文本: ```{text}```
"""

# 然后，我们调用`ChatPromptTemplatee.from_template()`函数将
# 上面的提示模版字符`template_string`转换为提示模版`prompt_template`

prompt_template = ChatPromptTemplate.from_template(template_string)

print(prompt_template)

customer_style = """正式普通话 \
用一个平静、尊敬的语气
"""

customer_email = """
有人违法使用我的电话信息，请马上注销，不然我将投诉12315/信息安全局以及曝光，请2小时马上答复我。
"""

# 使用提示模版
customer_messages = prompt_template.format_messages(
    style=customer_style, text=customer_email
)

print(customer_messages)

customer_response = chat(customer_messages)
print(customer_response.content)


input_variables=['style', 'text'] messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['style', 'text'], template='把由三个反引号分隔的文本翻译成一种{style}风格。文本: ```{text}```\n'))]
[HumanMessage(content='把由三个反引号分隔的文本翻译成一种正式普通话 用一个平静、尊敬的语气\n风格。文本: ```\n有人违法使用我的电话信息，请马上注销，不然我将投诉12315/信息安全局以及曝光，请2小时马上答复我。\n```\n')]


/opt/anaconda3/envs/.venv/lib/python3.9/site-packages/langchain_core/_api/deprecation.py:139: LangChainDeprecationWarning: The method `BaseChatModel.__call__` was deprecated in langchain-core 0.1.7 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(


尊敬的相关部门，我发现有人违法使用了我的电话信息，请立即注销。如果不立即处理，我将不得不向12315/信息安全局投诉并曝光此事。请在两小时内给予答复，谢谢。


#### 输出解释器
以上`customer_response`输出为 `str` 类型，如果想要方便的提取信息，需要使用`Langchain`中的输出解释器。

In [5]:
from langchain.output_parsers import ResponseSchema, StructuredOutputParser

template_string = """
对于以下文本，提取出以下信息：

语气：语气是否平和

语言风格：语言风格是否正式

使用以下键将输出格式化为 JSON：
语气
语言风格


文本：{text}

{format_instructions}
"""

prompt_template = ChatPromptTemplate.from_template(template_string)

response_schemas  = [
  ResponseSchema(name="语气", description="语气是否平和"),
  ResponseSchema(name="语言风格", description="语言风格是否正式")
]


output_parser = StructuredOutputParser.from_response_schemas(response_schemas)
format_instructions = output_parser.get_format_instructions()
print("输出格式规定：",format_instructions)



text = """
有人违法使用我的电话信息，请马上注销，不然我将投诉12315/信息安全局以及曝光，请2小时马上答复我。
"""

# 使用提示模版
message = prompt_template.format_messages(text=text, format_instructions=format_instructions)

customer_response = chat(message)
print("结果类型:", type(customer_response.content))
print(customer_response.content)

# output_dict = output_parser.parse(customer_response.content)
# print("解析后的结果类型:", type(output_dict))




输出格式规定： The output should be a markdown code snippet formatted in the following schema, including the leading and trailing "```json" and "```":

```json
{
	"语气": string  // 语气是否平和
	"语言风格": string  // 语言风格是否正式
}
```
结果类型: <class 'str'>
```json
{
	"语气": "激动"  // 语气是否平和
	"语言风格": "正式"  // 语言风格是否正式
}
```


### 储存 Memory
使用 LangChain 中的储存(Memory)模块时，它旨在保存、组织和跟踪整个对话的历史，从而为用户和模型之间的交互提供连续的上下文。

LangChain 提供了多种储存类型。其中:
* 缓冲区储存允许保留最近的聊天消息
* 摘要储存则提供了对整个对话的摘要。
* 实体储存则允许在多轮对话中保留有关特定实体的信息。

这些记忆组件都是模块化的，可与其他组件组合使用，从而增强机器人的对话管理能力。

主要的 4 种存储模块：
* 对话缓存储存 (ConversationBufferMemory）
* 对话缓存窗口储存 (ConversationBufferWindowMemory）
* 对话令牌缓存储存 (ConversationTokenBufferMemory）
* 对话摘要缓存储存 (ConversationSummaryBufferMemory）

#### 对话缓存存储
属于缓冲区储存器，用于存储对话上下文信息，包括历史消息、当前消息、历史消息的embedding等。

In [ ]:
from langchain.chains import ConversationChain
from langchain.chat_models import ChatOpenAI
from langchain.memory import ConversationBufferMemory

# 这里我们将参数temperature设置为0.0，从而减少生成答案的随机性。
# 如果你想要每次得到不一样的有新意的答案，可以尝试增大该参数。
llm = ChatOpenAI(temperature=0.0, api_key=api_key, base_url=base_url)
memory = ConversationBufferMemory()
memory.save_context({"input": "你好，我叫皮皮鲁"}, {"output": "你好啊，我叫鲁西西"})

# 新建一个 ConversationChain Class 实例
# verbose参数设置为True时，程序会输出更详细的信息，以提供更多的调试或运行时信息。
# 相反，当将verbose参数设置为False时，程序会以更简洁的方式运行，只输出关键的信息。
conversation = ConversationChain(llm=llm, memory = memory, verbose=True )

conversation.predict(input="你好, 我叫池")
conversation.predict(input="1+1 等于几")
conversation.predict(input="我叫什么")

memory.load_memory_variables({})

#### 对话窗口存储
对话缓存窗口储存只保留一个窗口大小的对话。它只使用最近的n次交互。这可以用于保持最近交互的滑动窗口，以便缓冲区不会过大。

In [ ]:
from langchain.memory import ConversationBufferWindowMemory

# k=1表明只保留一个对话记忆
memory = ConversationBufferWindowMemory(k=1)  

llm = ChatOpenAI(temperature=0.0, api_key=api_key, base_url=base_url)
conversation = ConversationChain(llm=llm, memory=memory, verbose=False  )

print("第一轮对话：")
print(conversation.predict(input="你好, 我叫池"))

print("第二轮对话：")
print(conversation.predict(input="我叫什么名字？"))


#### 对话字符缓存存储
使用对话字符缓存记忆，内存将限制保存的token数量。如果字符数量超出指定数目，它会切掉这个对话的早期部分 以保留与最近的交流相对应的字符数量，但不超过字符限制。

In [ ]:
from langchain.llms import OpenAI
from langchain.memory import ConversationTokenBufferMemory
memory = ConversationTokenBufferMemory(llm=llm, max_token_limit=30)
memory.save_context({"input": "朝辞白帝彩云间，"}, {"output": "千里江陵一日还。"})
memory.save_context({"input": "两岸猿声啼不住，"}, {"output": "轻舟已过万重山。"})
memory.load_memory_variables({})

### 模型链 Chains
链（Chains）通常将大语言模型（LLM）与提示（Prompt）结合在一起，基于此，我们可以对文本或数据进行一系列操作。链（Chains）可以一次性接受多个输入。

#### 简单顺序链
预定义顺序执行其链接的链，其中每个步骤都有一个输入/输出，一个步骤的输出是下一个步骤的输入。

In [6]:
import warnings
warnings.filterwarnings('ignore')

from langchain.chat_models import ChatOpenAI 
from langchain.prompts import ChatPromptTemplate  
from langchain.chains import LLMChain,SimpleSequentialChain


llm = ChatOpenAI(temperature=0.0, api_key=api_key, base_url=base_url)

# prompt = ChatPromptTemplate.from_template("描述制造{product}的一个公司的最佳名称是什么?")

# chain = LLMChain(llm=llm, prompt=prompt)


# 提示模板 1 ：这个提示将接受产品并返回最佳名称来描述该公司
first_prompt = ChatPromptTemplate.from_template(   
    "描述制造{product}的一个公司的最好的名称是什么"
)
chain_one = LLMChain(llm=llm, prompt=first_prompt)

# 提示模板 2 ：接受公司名称，然后输出该公司的长为20个单词的描述
second_prompt = ChatPromptTemplate.from_template(   
    "写一个10字的描述对于下面这个\
    公司：{company_name}的"
)
chain_two = LLMChain(llm=llm, prompt=second_prompt)

overall_simple_chain = SimpleSequentialChain(chains=[chain_one, chain_two], verbose=True)

product = "大号床单套装"
overall_simple_chain.run(product)





> Entering new SimpleSequentialChain chain...
"Royal Linens"
Royal Linens：提供高质量的床上用品和家居装饰产品。

> Finished chain.


'Royal Linens：提供高质量的床上用品和家居装饰产品。'

#### 多输入、输出顺序链
当只有一个输入和一个输出时，简单顺序链（SimpleSequentialChain）即可实现。当有多个输入或多个输出时，我们则需要使用顺序链（SequentialChain）来实现。

In [38]:
import pandas as pd
from langchain.chains import SequentialChain
from langchain.chat_models import ChatOpenAI    #导入OpenAI模型
from langchain.prompts import ChatPromptTemplate   #导入聊天提示模板
from langchain.chains import LLMChain    #导入LLM链。

llm = ChatOpenAI(temperature=0.9, api_key=api_key, base_url=base_url)

创建 4 个子链

In [39]:
#子链1
# prompt模板 1: 翻译成英语（把下面的review翻译成英语）
first_prompt = ChatPromptTemplate.from_template(
    "把下面的水果review翻译成英文:"
    "\n\n{Review}"
)
# chain 1: 输入：Review    输出：英文的 Review
chain_one = LLMChain(llm=llm, prompt=first_prompt, output_key="English_Review")

#子链2
# prompt模板 2: 用一句话总结下面的 review
second_prompt = ChatPromptTemplate.from_template(
    "请你用一句话来描述下面的review:"
    "\n\n{English_Review}"
)
# chain 2: 输入：英文的Review   输出：总结
chain_two = LLMChain(llm=llm, prompt=second_prompt, output_key="summary")


#子链3
# prompt模板 3: 下面review使用的什么语言
third_prompt = ChatPromptTemplate.from_template(
    "下面的review使用的什么语言:\n\n{Review}"
)
# chain 3: 输入：Review  输出：语言
chain_three = LLMChain(llm=llm, prompt=third_prompt, output_key="language")


#子链4
# prompt模板 4: 使用特定的语言对下面的总结写一个后续回复
fourth_prompt = ChatPromptTemplate.from_template(
    "使用特定的语言对下面的总结写一个后续回复:"
    "\n\n总结: {summary}\n\n语言: {language}"
)
# chain 4: 输入： 总结, 语言    输出： 后续回复
chain_four = LLMChain(llm=llm, prompt=fourth_prompt, output_key="followup_message")


组合四个子链

In [40]:
#输入：review    
#输出：英文review，总结，后续回复 
overall_chain = SequentialChain(
    chains=[chain_one, chain_two, chain_three, chain_four],
    input_variables=["Review"],
    output_variables=["English_Review", "summary","followup_message"],
    verbose=True
)
review = "苹果"
overall_chain(review)




> Entering new SequentialChain chain...

> Finished chain.


{'Review': '苹果',
 'English_Review': 'Apple\n\nThe apple is a popular and versatile fruit that is enjoyed by people of all ages. It is crisp and juicy, with a sweet and slightly tart flavor. Apples can be eaten on their own as a healthy snack, or used in a variety of dishes such as pies, crisps, and salads. They are also a good source of fiber and vitamin C, making them a nutritious choice for any diet. Overall, apples are a delicious and refreshing fruit that can be enjoyed year-round.',
 'summary': 'Apples are a delicious and versatile fruit that can be enjoyed in many different ways and are a nutritious choice for any diet.',
 'followup_message': '谢谢你的总结!苹果是一种美味多样、可以用多种方式享用的水果，不仅适合任何饮食，而且营养丰富。我也喜欢吃苹果，它们是我日常饮食的重要组成部分。你有什么喜欢的苹果食谱吗？分享给我们吧!'}

#### 路由链
一个相当常见但基本的操作是根据输入将其路由到一条链，具体取决于该输入到底是什么。如果你有多个子链，每个子链都专门用于特定类型的输入，那么可以组成一个路由链，它首先决定将它传递给哪个子链，然后将它传递给那个链。

In [41]:
from langchain.chains.router import MultiPromptChain  #导入多提示链
from langchain.chains.router.llm_router import LLMRouterChain,RouterOutputParser
from langchain.prompts import PromptTemplate

llm = ChatOpenAI(temperature=0.9, api_key=api_key, base_url=base_url)


首先，我们定义提示适用于不同场景下的提示模板。

In [42]:
# 中文
#第一个提示适合回答物理问题
physics_template = """你是一个非常聪明的物理专家。 \
你擅长用一种简洁并且易于理解的方式去回答问题。\
当你不知道问题的答案时，你承认\
你不知道.

这是一个问题:
{input}"""


#第二个提示适合回答数学问题
math_template = """你是一个非常优秀的数学家。 \
你擅长回答数学问题。 \
你之所以如此优秀， \
是因为你能够将棘手的问题分解为组成部分，\
回答组成部分，然后将它们组合在一起，回答更广泛的问题。

这是一个问题：
{input}"""


#第三个适合回答历史问题
history_template = """你是以为非常优秀的历史学家。 \
你对一系列历史时期的人物、事件和背景有着极好的学识和理解\
你有能力思考、反思、辩证、讨论和评估过去。\
你尊重历史证据，并有能力利用它来支持你的解释和判断。

这是一个问题:
{input}"""


#第四个适合回答计算机问题
computerscience_template = """ 你是一个成功的计算机科学专家。\
你有创造力、协作精神、\
前瞻性思维、自信、解决问题的能力、\
对理论和算法的理解以及出色的沟通技巧。\
你非常擅长回答编程问题。\
你之所以如此优秀，是因为你知道  \
如何通过以机器可以轻松解释的命令式步骤描述解决方案来解决问题，\
并且你知道如何选择在时间复杂性和空间复杂性之间取得良好平衡的解决方案。

这还是一个输入：
{input}"""


构建聊天模型为每个模板命名，并给出具体描述。类似于我们软件开发中配置的页面路由

In [43]:
# 中文
prompt_infos = [
    {
        "名字": "物理学", 
        "描述": "擅长回答关于物理学的问题", 
        "提示模板": physics_template
    },
    {
        "名字": "数学", 
        "描述": "擅长回答数学问题", 
        "提示模板": math_template
    },
    {
        "名字": "历史", 
        "描述": "擅长回答历史问题", 
        "提示模板": history_template
    },
    {
        "名字": "计算机科学", 
        "描述": "擅长回答计算机科学问题", 
        "提示模板": computerscience_template
    }
]


基于模板信息创建相应目标链：目标链是由路由链调用的链，每个目标链都是一个语言模型链，构建了一个链的映射 ，根据链的名称找 chain

In [44]:
destination_chains = {}
for p_info in prompt_infos:
    name = p_info["名字"]
    prompt_template = p_info["提示模板"]
    prompt = ChatPromptTemplate.from_template(template=prompt_template)
    chain = LLMChain(llm=llm, prompt=prompt)
    destination_chains[name] = chain  
    
destinations = [f"{p['名字']}: {p['描述']}" for p in prompt_infos]
destinations_str = "\n".join(destinations)

# print(destinations_str)

# 创建默认目标链，当路由器无法决定使用哪个子链时调用的链。
default_prompt = ChatPromptTemplate.from_template("{input}")
default_chain = LLMChain(llm=llm, prompt=default_prompt)


多路由提示模板

In [45]:
# 多提示路由模板
MULTI_PROMPT_ROUTER_TEMPLATE = """给语言模型一个原始文本输入，\
让其选择最适合输入的模型提示。\
系统将为您提供可用提示的名称以及最适合改提示的描述。\
如果你认为修改原始输入最终会导致语言模型做出更好的响应，\
你也可以修改原始输入。


<< 格式 >>
返回一个带有JSON对象的markdown代码片段，该JSON对象的格式如下：
```json
{{{{
    "destination": 字符串 \ 使用的提示名字或者使用 "DEFAULT"
    "next_inputs": 字符串 \ 原始输入的改进版本
}}}}



记住：“destination”必须是下面指定的候选提示名称之一，\
或者如果输入不太适合任何候选提示，\
则可以是 “DEFAULT” 。
记住：如果您认为不需要任何修改，\
则 “next_inputs” 可以只是原始输入。

<< 候选提示 >>
{destinations}

<< 输入 >>
{{input}}

<< 输出 (记得要包含 ```json)>>

样例:
<< 输入 >>
"什么是黑体辐射?"
<< 输出 >>
```json
{{{{
    "destination": 字符串 \ 使用的提示名字或者使用 "DEFAULT"
    "next_inputs": 字符串 \ 原始输入的改进版本
}}}}

"""


1. 构建路由链
2. 创建整体链路
3. 提问

In [46]:
router_template = MULTI_PROMPT_ROUTER_TEMPLATE.format(
    destinations=destinations_str
)
router_prompt = PromptTemplate(
    template=router_template,
    input_variables=["input"],
    output_parser=RouterOutputParser(),
)

router_chain = LLMRouterChain.from_llm(llm, router_prompt)

#多提示链
chain = MultiPromptChain(router_chain=router_chain,    #l路由链路
                         destination_chains=destination_chains,   #目标链路
                         default_chain=default_chain,      #默认链路
                         verbose=True   
                        )

chain.run("什么是黑体辐射？")
chain.run("2+2等于多少？")




> Entering new MultiPromptChain chain...
物理学: {'input': '黑体辐射是指理想化的物体在一定温度下所发出的辐射，其辐射强度与波长和温度有关。'}
> Finished chain.


> Entering new MultiPromptChain chain...
数学: {'input': '2 plus 2 equals how much?'}
> Finished chain.


'2 plus 2 equals 4.'

### 基于文档的回答
与仅依赖模型预训练知识不同，这种方法可以进一步整合用户自有数据，实现更加个性化和专业的问答服务。例如,我们可以收集某公司的内部文档、产品说明书等文字资料，导入问答系统中。然后用户针对这些文档提出问题时，系统可以先在文档中检索相关信息，再提供给语言模型生成答案。

这样，语言模型不仅利用了自己的通用知识，还可以充分运用外部输入文档的专业信息来回答用户问题，显著提升答案的质量和适用性。构建这类基于外部文档的问答系统，可以让语言模型更好地服务于具体场景，而不是停留在通用层面。这种灵活应用语言模型的方法值得在实际使用中推广。

#### 使用向量储存查询
##### 导入数据

In [ ]:
from langchain.chains import RetrievalQA  #检索QA链，在文档上进行检索
from langchain.chat_models import ChatOpenAI  #openai模型
from langchain.document_loaders import CSVLoader #文档加载器，采用csv格式存储
from langchain.vectorstores import DocArrayInMemorySearch  #向量存储
from IPython.display import display, Markdown #在jupyter显示信息的工具
import pandas as pd

file = './data/OutdoorClothingCatalog_1000.csv'

# 使用langchain文档加载器对数据进行导入
loader = CSVLoader(file_path=file)

# 使用pandas导入数据，用以查看
data = pd.read_csv(file,usecols=[1, 2])
data.head()


##### 基本文档加载器创建向量存储

In [ ]:
#导入向量存储索引创建器
from langchain.indexes import VectorstoreIndexCreator 
from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores import DocArrayInMemorySearch  #向量存储

# # 创建一个嵌入模型实例
embedding_model = OpenAIEmbeddings(api_key=api_key, base_url=base_url)

# 创建指定向量存储类, 创建完成后，从加载器中调用, 通过文档加载器列表加载
index = VectorstoreIndexCreator(vectorstore_cls=DocArrayInMemorySearch,embedding=embedding_model).from_loaders([loader])



##### 查询创建的向量存储

In [ ]:
query ="请用markdown表格的方式列出所有具有防晒功能的衬衫，对每件衬衫描述进行总结"

#使用索引查询创建一个响应，并传入这个查询
response = index.query(query)

#查看查询返回的内容
display(Markdown(response))


#### 结合表征模型和向量存储
由于语言模型的上下文长度限制，直接处理长文档具有困难。为实现对长文档的问答，我们可以引入向量嵌入(Embeddings)和向量存储(Vector Store)等技术

首先，使用文本嵌入(Embeddings)算法对文档进行向量化，使语义相似的文本片段具有接近的向量表示。其次，将向量化的文档切分为小块，存入向量数据库，这个流程正是创建索引(index)的过程。

In [31]:
from langchain.document_loaders import CSVLoader #文档加载器，采用csv格式存储
from langchain.text_splitter import RecursiveCharacterTextSplitter

# 1. 创建一个文档加载器，通过csv格式加载
file = './data/product_data.csv'
loader = CSVLoader(file_path=file)
docs = loader.load()

# 2. 拆分为更小的块
# text_splitter = RecursiveCharacterTextSplitter(chunk_size=100000, chunk_overlap=10)
# splits = text_splitter.split_documents(docs)

#查看单个文档，每个文档对应于CSV中的一行数据
docs[0]


Document(page_content="product_name: 全自动咖啡机\ndescription: 规格:\n大型 - 尺寸：13.8'' x 17.3''。\n中型 - 尺寸：11.5'' x 15.2''。\n\n为什么我们热爱它:\n这款全自动咖啡机是爱好者的理想选择。 一键操作，即可研磨豆子并沏制出您喜爱的咖啡。它的耐用性和一致性使它成为家庭和办公室的理想选择。\n\n材质与护理:\n清洁时只需轻擦。\n\n构造:\n由高品质不锈钢制成。\n\n其他特性:\n内置研磨器和滤网。\n预设多种咖啡模式。\n在中国制造。\n\n有问题？ 请随时联系我们的客户服务团队，他们会解答您的所有问题。", metadata={'source': './data/product_data.csv', 'row': 0})

In [33]:
# 3. 向量存储
from langchain.embeddings import OpenAIEmbeddings 

embeddings = OpenAIEmbeddings(api_key=api_key, base_url=base_url)

#因为文档比较短了，所以这里不需要进行任何分块,可以直接进行向量表征
#使用初始化OpenAIEmbedding实例上的查询方法embed_query为文本创建向量表征
embed = embeddings.embed_query("咖啡机的好处")

#查看得到向量表征的长度
print("\n\033[32m向量表征的长度: \033[0m \n", len(embed))

#每个元素都是不同的数字值，组合起来就是文本的向量表征
print("\n\033[32m向量表征前5个元素: \033[0m \n", embed[:5])



向量表征的长度:  
 1536

向量表征前5个元素:  
 [-0.012509773620940318, -0.011175488342259047, 0.02782357865910436, -0.03673687716093936, -0.02529046896160299]


In [35]:
from langchain.vectorstores import DocArrayInMemorySearch  #向量存储

# 将刚才创建文本向量表征(embeddings)存储在向量存储(vector store)中
# 使用DocArrayInMemorySearch类的from_documents方法来实现
# 该方法接受文档列表以及向量表征模型作为输入，docs 可以替换为 splits
retriever = DocArrayInMemorySearch.from_documents(docs, embeddings)

query = "请推荐一个咖啡机"
#使用上面的向量存储来查找与传入查询类似的文本，得到一个相似文档列表
docs = retriever.similarity_search(query)
print("\n\033[32m返回文档的个数: \033[0m \n", len(docs))
print("\n\033[32m第一个文档: \033[0m \n", docs[0])
print("\n\033[32m第二个文档: \033[0m \n", docs[1])
print("\n\033[32m第三个文档: \033[0m \n", docs[2])


返回文档的个数:  
 4

第一个文档:  
 page_content="product_name: 全自动咖啡机\ndescription: 规格:\n大型 - 尺寸：13.8'' x 17.3''。\n中型 - 尺寸：11.5'' x 15.2''。\n\n为什么我们热爱它:\n这款全自动咖啡机是爱好者的理想选择。 一键操作，即可研磨豆子并沏制出您喜爱的咖啡。它的耐用性和一致性使它成为家庭和办公室的理想选择。\n\n材质与护理:\n清洁时只需轻擦。\n\n构造:\n由高品质不锈钢制成。\n\n其他特性:\n内置研磨器和滤网。\n预设多种咖啡模式。\n在中国制造。\n\n有问题？ 请随时联系我们的客户服务团队，他们会解答您的所有问题。" metadata={'source': './data/product_data.csv', 'row': 0}

第二个文档:  
 page_content="product_name: 自动咖啡机\ndescription: 规格:\n尺寸：12'' x 8'' x 14''。\n\n为什么我们热爱它:\n我们的自动咖啡机可以轻松制作美味的咖啡，只需按一下钮，就能享受到新鲜的咖啡。\n\n材质与护理:\n可以用湿布清洁外部，内部配件可以拆卸清洗。\n\n构造:\n由不锈钢和塑料制成。\n\n其他特性:\n可调节咖啡浓度。\n在意大利制造。\n\n有问题？请随时联系我们的客户服务团队，他们会解答您的所有问题。" metadata={'source': './data/product_data.csv', 'row': 17}

第三个文档:  
 page_content='product_name: 陶瓷保温杯\ndescription: 规格:\n容量：350ml。\n\n为什么我们热爱它:\n我们的陶瓷保温杯设计优雅，保温效果好，是办公室和户外活动的理想伴侣。\n\n材质与护理:\n可以手洗或洗碗机清洗。\n\n构造:\n由高品质陶瓷和不锈钢制成。\n\n其他特性:\n有多种颜色可选。\n在中国制造。\n\n有问题？请随时联系我们的客户服务团队，他们会解答您的所有问题。' metadata={'source': './data/product_data.csv', 'row': 8}


In [36]:
#导入大语言模型, 这里使用默认模型gpt-3.5-turbo会出现504服务器超时，
#因此使用gpt-3.5-turbo-0301
llm = ChatOpenAI(model_name="gpt-3.5-turbo",temperature = 0.0) 

#合并获得的相似文档内容
qdocs = "".join([docs[i].page_content for i in range(len(docs))])  


#将合并的相似文档内容后加上问题（question）输入到 `llm.call_as_llm`中
#这里问题是：以Markdown表格的方式列出所有具有防晒功能的衬衫并总结 
response = llm.call_as_llm(f"{qdocs}问题：请用markdown表格的方式列出所有咖啡机，对每个咖啡机描述进行总结") 

display(Markdown(response))


| 产品名称      | 描述总结                                                                                   |
|--------------|------------------------------------------------------------------------------------------|
| 全自动咖啡机 | 一键操作，研磨豆子并沏制出喜爱的咖啡，耐用性和一致性，适合家庭和办公室使用。高品质不锈钢制造。内置研磨器和滤网，多种咖啡模式，中国制造。 |
| 自动咖啡机   | 按钮操作，制作美味咖啡，易清洁，不锈钢和塑料制造，可调节咖啡浓度，意大利制造。               |